In [5]:
import pandas as pd
import re
import jellyfish

# Définition du fichier d'alignement à vérifier
input_file_path = "valid-align-AI-P2-20231031.tsv"

# Définition du fichier de sortie pour les cas suspects
output_file_root = "a-verifier_part2"

# Définition des colonnes contenant les formes sources et cible à comparer
source_form_colname = "forme principale source"
target_form_colname = "forme principale cible"
align_type_colname = "décision d'alignement"
target_colname = "réservoir cible"
target_candidatenr_colname = "nombre de candidats"

# Types d'alignement à vérifier
valid_types = {'externe', 'manuel'}

# Le fichier à vérifier est chargé dans une dataframe
df = pd.read_csv(input_file_path, sep='\t', dtype = str)
print(df.size)

# On retire les types d'alignements ignorés ainsi que les non-alignements
df_filtered = df[(df[align_type_colname].isin(valid_types)) & (df[target_colname] == 'idref') & (df[target_candidatenr_colname] == '0')]
print(df_filtered.size)
display(df_filtered.head())

2812992
114609


,réservoir source,id source,forme principale source,arbitre,date d'arbitrage,niveau de confiance,commentaire,décision d'alignement,nombre de candidats,score algo max,...,id cible,forme principale cible,type de cible 2,réservoir cible 2,id cible 2,forme principale cible 2,type de cible 3,réservoir cible 3,id cible 3,forme principale cible 3
0,rnv-nz-auth-atc,981023132827302851,"Daniel, Marie",7035,12/09/2023 - 14:29:06.065,Certain,NaN,manuel,0,NaN,...,134948718,"Daniel, Marie",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,rnv-nz-auth-atc,981023280283502851,"Zacharie, de Lisieux",8153,16/08/2022 - 23:04:59.469,Certain,NaN,manuel,0,NaN,...,029184541,Zacharie de Lisieux 1596-1661,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,rnv-nz-auth-atc,981023280312002851,"Savoye, Marie-Catherine",NaN,NaN,NaN,NaN,externe,0,NaN,...,074313371,"Savoye, Veuve d'Étienne-François 171.-1781",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,rnv-nz-auth-atc,981023280313602851,"Beeby-Thompson, A",NaN,NaN,NaN,NaN,externe,0,NaN,...,147520681,"Thompson, Arthur Beeby",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,rnv-nz-auth-atc,981023280454502851,"Pericot García, Luis",NaN,NaN,NaN,NaN,externe,0,NaN,...,032437579,"Pericot, Luis",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Comparaison des dates, s'il y en a
output_date_suffix = "_dates.xlsx"
output_other_suffix = "_erreurs.xlsx"

wrong_dates_df = pd.DataFrame()
misc_errors_df = pd.DataFrame()

# Fonction de comparaison des dates
def date_compare(source_date,target_date):
    source_date = str(source_date)
    target_date = str(target_date)
    # Ignorer les points d'interrogation
    source_date = source_date.replace('?','')
    target_date = target_date.replace('?','')
    # Retirer le zéro en début de date
    if source_date.startswith('0'):
        source_date = source_date[1:0]
    if target_date.startswith('0'):
        target_date = target_date[1:0]
    # Si l'une des dates à comparer est vide, on passe
    if ((source_date == '') | (target_date == '')):
        return False
    # Si l'une des dates comporte des points, ne comparer que les chiffres entre eux
    if (('.' in source_date) | ('.' in target_date)):
        #print('Comparaison avec points: ' + source_date + ' et ' + target_date)
        for char in range(0,len(source_date)):
            if ((source_date[char] != '.') & (target_date[char] != '.') & (source_date[char] != target_date[char])):
                #print('Je pense que ' + source_date[char] + ' != ' + target_date[char])
                return True
        return False
    if (source_date != target_date):
        return True
    else:
        return False

# Expression régulière pour trouver les dates
date_pattern = r'(?:ca\.|fl\.)?([0-9\.]{4}|\?)-?([0-9\.]{4}|\?)?'

#df_filtered = df_filtered.reset_index()  # make sure indexes pair with number of rows

for index, row in df_filtered.iterrows():
    if not ((row[source_form_colname] == '') | (row[target_form_colname] == '') | pd.isnull(row[source_form_colname]) | pd.isnull(row[target_form_colname])):
        source_dates = re.findall(date_pattern, row[source_form_colname])
        target_dates = re.findall(date_pattern, row[target_form_colname])
        if ((len(source_dates) > 0) & (len(target_dates) > 0)):
            if (date_compare(source_dates[0][0],target_dates[0][0]) | date_compare(source_dates[0][1],target_dates[0][1])):
                # Cas potentiel de dates qui ne correspondent pas
                wrong_dates_df = wrong_dates_df.append(row)
    else:
        # Il manque un des champs à comparer, autre erreur
        misc_errors_df = misc_errors_df.append(row)

wrong_dates_df.to_excel(output_file_root + output_date_suffix)
misc_errors_df.to_excel(output_file_root + output_other_suffix)
#display(wrong_dates_df.head(50))

In [4]:
# Identifier les entrées très différentes

output_comp_suffix = "_comparaison.xlsx"

comp_df = pd.DataFrame()

comp_margin = 15
df_head = df_filtered.head(100)

# Expression régulière pour exclure les dates
text_pattern = r'([A-Za-z, ]+)'

for index, row in df_filtered.iterrows():
    source_text = re.search(text_pattern,row[source_form_colname]).group()
    target_text = re.search(text_pattern,row[target_form_colname]).group()
    distance = jellyfish.levenshtein_distance(source_text,target_text)
    if distance > comp_margin:
        comp_df = comp_df.append(row)

comp_df.to_excel(output_file_root + output_comp_suffix)